In [82]:
import tool
import scipy
import pandas as pd
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, fixed
import warnings
warnings.filterwarnings('ignore')

In [83]:
"""
Data initialisation
"""

asset_index = pd.read_csv("data/AIDX.csv", encoding='gbk')

index_list = asset_index['S_IRDCODE'].drop_duplicates().sample(np.random.randint(*(5, 10))).tolist()

asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
asset_index.sort_values(by='TRADE_DT', inplace=True)
asset_index.set_index('TRADE_DT', inplace=True)
asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list].dropna()

print(index_list)
print(asset_index)

['h30258.CSI', '000970.CSI', '931590.CSI', 'L01177.CSI', 'h50014.SH', 'h00964.CSI', 'CN2618.CNI', 'h11159.CSI']
S_IRDCODE   h30258.CSI  000970.CSI  931590.CSI  L01177.CSI  h50014.SH  \
TRADE_DT                                                                
2023-01-03   3198.2107   1470.0032  22533.9280   4266.7012  3213.5575   
2023-01-04   3218.3648   1475.9357  22450.4631   4224.3211  3206.2781   
2023-01-05   3225.2544   1487.0431  22736.2518   4268.1777  3234.8556   
2023-01-06   3186.6049   1483.8890  22863.0196   4231.5040  3245.0828   
2023-01-09   3207.2135   1491.2535  23010.0439   4271.5311  3290.3362   
...                ...         ...         ...         ...        ...   
2023-12-20   4341.2806   1411.5728  19966.2309   4422.7680  3282.4433   
2023-12-21   4341.2806   1411.5728  19966.2309   4422.7680  3282.4433   
2023-12-22   4341.2806   1411.5728  19966.2309   4422.7680  3282.4433   
2023-12-25   4341.2806   1411.5728  19966.2309   4422.7680  3282.4433   
2023-12-26  

In [84]:
"""
Parameters
"""
BACKTEST_DAY = 30
MODEL_TYPE = 'RB' # MVO, RP, BL, RB
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate
REBALANCE_DAYS = 30

N = len(index_list)
index_min_weight = [0 for _ in range(N)]
index_max_weight = [1 for _ in range(N)]
WEIGHT_CONSTRAINTS = list(zip(index_min_weight, index_max_weight))

In [85]:
"""
Rebalancing
"""
def rebalance(asset_index, T):
    predicts = []
    actuals = []
    realities = []
    
    for i in range(T, len(asset_index), T):
        
        if i+T >= len(asset_index):
            break
        
        historical_data = asset_index[i-T:i]
        future_data = asset_index[i:i+T]
        
        predict, actual = tool.evaluate(historical_data, future_data, WEIGHT_CONSTRAINTS, MODEL_TYPE, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        reality = tool.check([1 / N for _ in range(N)], future_data, RISK_FREE_RATE)  # equally weighed
        realities.append(reality)
    
    return predicts, actuals, realities

predicts, actuals, realities = rebalance(asset_index, REBALANCE_DAYS)
print(predicts)
print(actuals)
print(realities)

[(0.6801872028753819, 268.08187101403917, 0.002462632778479857), (5.854691731421724e-17, 238.91012589719875, -8.371348817841984e-05), (1.3877787807814457e-17, 66.6835299850663, -0.00029992413425742405), (2.0817739931858625e-17, 155.252678731554, -0.0001288222539115206), (2.4286128563351297e-17, 109.3808256521437, -0.00018284740383661574), (6.938894041681362e-18, 129.03740802646487, -0.0001549938138551117), (-4.1633365090697756e-17, 26.071924288886315, -0.0007671087020042263)]
[(0.32665802166009805, 158.4875894617065, 0.0019349024280175087), (0.08413931726814415, 264.49469980007916, 0.00024249755218771666), (-0.21435637548221304, 76.88830063079823, -0.0030480108619846346), (-0.08443132546910805, 132.6534889881411, -0.0007872489918334825), (-0.33409968655094824, 273.0063629695603, -0.0012970382180814946), (-0.16886820399696023, 108.38112196103627, -0.0017426300870446755), (2.533475722963517e-25, 3.601743312125242e-13, -55528665612.20548)]
[(0.5353635438538611, 101.74662824767216, 0.00506

In [88]:
"""
Output
"""

def display(L1, L2, L3, normalize=False, lines_to_show=None):
    line_names = ['r1', 'v1', 's1', 'r2', 'v2', 's2', 'r3', 'v3', 's3']
    line_styles = {
        'r1': 'r-', 'v1': 'r--', 's1': 'r:',
        'r2': 'b-', 'v2': 'b--', 's2': 'b:',
        'r3': 'g-', 'v3': 'g--', 's3': 'g:'
    }

    a1, b1, c1 = zip(*L1)
    a2, b2, c2 = zip(*L2)
    a3, b3, c3 = zip(*L3)

    lines = {'r1': a1, 'v1': b1, 's1': c1, 
             'r2': a2, 'v2': b2, 's2': c2,
             'r3': a3, 'v3': b3, 's3': c3}

    def normalize_data(data):
        return (data - np.mean(data)) / np.std(data)

    if normalize:
        lines = {name: normalize_data(data) for name, data in lines.items()}

    plt.figure(figsize=(10, 6))
    for line in lines_to_show:
        plt.plot(lines[line], line_styles[line], label=line, marker='o')
    
    plt.title("R: Return\tV: Volatility\tS: Sharpe Ratio\n1: Predicted\t2: Actual\t3: Reality\nButton 'normalize': Normalise each line")
    plt.legend()
    
    for pair in [('r1', 'r2'), ('v1', 'v2'), ('s1', 's2')]:
        if pair[0] in lines_to_show and pair[1] in lines_to_show:
            corr, _ = scipy.stats.spearmanr(lines[pair[0]], lines[pair[1]])
            print(f"Spearman correlation between {pair[0]} and {pair[1]}: {corr:.2f}")
    
    plt.show()


# @interact
interact(display, 
         L1=fixed(predicts), 
         L2=fixed(actuals),
         L3=fixed(realities), 
         normalize=True, 
         lines_to_show=SelectMultiple(options=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'],
                                      value=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'], 
                                      description='Lines'))



interactive(children=(Checkbox(value=True, description='normalize'), SelectMultiple(description='Lines', index…

<function __main__.display(L1, L2, L3, normalize=False, lines_to_show=None)>